## Import libaries

In [2]:
import os

if os.getcwd().endswith('/notebooks'):
    os.chdir('..')

assert os.path.exists('./src'), f"[ERROR] The path src not detected in the current directory '{os.getcwd()}'."

print(f'[INFO] Current Directory: "{os.getcwd()}".')

if not os.path.exists("images"):
    os.mkdir("images")

[INFO] Current Directory: "/home/ferdi/topic-modeling-advancements".


In [3]:
%load_ext autoreload
%autoreload 2
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer

from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.dataset.dataset import Dataset
from octis.models.LDA import LDA

import time
import pandas as pd
import numpy as np
import zlib
import pickle as pkl
import sklearn
import pandas as pd
from tabulate import tabulate
import kaleido

## Preencode data with embeddings
from sentence_transformers import SentenceTransformer
## Load data with datalaoder from src
from src.utils import load_documents
from src.bertopic_runner import Trainer
from umap import UMAP
from hdbscan import HDBSCAN

2022-07-20 11:21:11.864820: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-20 11:21:11.864957: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Parametric run: BERTopic

In [21]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/ferdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ferdi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ferdi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

If we set hdbscan_args to None, Kmeans is used for clustering. That way we can enforce having no noise clusters/documents.
As a consequence we also have no assignment scores since KMeans does not produce any (set as default to 1).

In [13]:
configs = {
    'dataset': 'crisis_toy',
    'preprocessing_funcs': [
        'to_lowercase',
        'standardize_accented_chars',
        'remove_url',
        'expand_contractions',
        'remove_mentions',
        'remove_hashtags',
        'remove_new_lines',
        'keep_only_alphabet',
        # 'remove_extra_spaces',
        'remove_english_stop_words',
        'lemmatize_noun'
    ],
    'algorithm': 'bertopic',
    'algorithm_args': {
        "embedding_model": "all-MiniLM-L6-v2",
        "top_n_words": 10,
        "n_gram_range_tuple": (1, 1),
        ## Both the same as below
        "min_docs_per_topic": 15,
        "number_topics": 4,
         ## Assign almost all docs to a topic
         #"no_noise": True,
         #"prob_threshold": 0.01,
         ## Setting min_samples to reduce #docs classified as noise/in topic -1
        "cluster_model": "kmeans" #"hdbscan"or "kmeans" then we do not need hdbscan_args; if used are ignored
        "hdbscan_args": {
                    "min_cluster_size": 15,
                    "metric":'euclidean',
                    "cluster_selection_method": 'eom',
                    "prediction_data": True,
                    #"min_samples": 15
         },    
        "umap_args": {
                    "n_neighbors": 15,
                    "n_components": 5,
                    "min_dist": 0.0,
                    "metric": 'cosine',
                    "low_memory": False,
                    "random_state": 42
        }
    }
}

SyntaxError: invalid syntax (1948206967.py, line 29)

In [ ]:
##Maybe parallelism disable?
#import os
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

from src import preprocessor
from src.utils import load_documents

docs, labels = load_documents(dataset=configs['dataset'])

if 'preprocessing_funcs' in configs:
    docs = preprocessor.run(data=docs, prep_functions=configs['preprocessing_funcs'])
    
algorithm_args = configs['algorithm_args']
algorithm_args.update(data_name=configs['dataset'],docs=docs,labels=labels)
print(f'Running with {algorithm_args["number_topics"]} topics')

if configs['algorithm'] == 'bertopic':
    # Encode data with embedding model
    model = SentenceTransformer(algorithm_args['embedding_model'])
    embeddings = model.encode(docs, show_progress_bar=True)
    
    trainer = Trainer(dataset = configs['dataset'],
                      model_name = configs['algorithm'],
                      params = algorithm_args,
                      topk = algorithm_args["top_n_words"],
                      bt_embeddings = embeddings,
                      )
    
    model, df_output_doc_topic, df_output_topic_word = trainer.train()
    
    from src.evaluator import compute_topic_scores
    df_output_topic_word = compute_topic_scores(df_output_doc_topic, df_output_topic_word)

In [14]:
df_output_topic_word

,run_id,method,method_specific_params,dataset,num_given_topics,reduced,topic_num,topic_size,topic_words,word_scores,num_detected_topics,num_final_topics,duration_secs,diversity_unique,diversity_inv_rbo,coherence_npmi,coherence_v,rand_index
0,1658309733,lda-bert,"{'embedding_model': 'all-MiniLM-L6-v2', 'numbe...",crisis_toy,3,False,0,149,"[smoke, debbie, cyclone, wildfire, earthquake,...","[0.03574329813160033, 0.03330625507717303, 0.0...",3,3,10.295372,0.666667,0.629645,-0.30072,0.316743,0.64121
1,1658309733,lda-bert,"{'embedding_model': 'all-MiniLM-L6-v2', 'numbe...",crisis_toy,3,False,1,116,"[flood, wildfire, cyclone, rain, debbie, smoke...","[0.052525252525252523, 0.01616161616161616, 0....",3,3,10.295372,0.666667,0.629645,-0.30072,0.316743,0.64121
2,1658309733,lda-bert,"{'embedding_model': 'all-MiniLM-L6-v2', 'numbe...",crisis_toy,3,False,2,131,"[earthquake, flood, debbie, cyclone, felt, old...","[0.06908267270668177, 0.03737259343148358, 0.0...",3,3,10.295372,0.666667,0.629645,-0.30072,0.316743,0.64121


## Parametric Run: LDA-BERT

In [10]:
configs = {
    'dataset': 'crisis_toy',
    'preprocessing_funcs': [
        'to_lowercase',
        'standardize_accented_chars',
        'remove_url',
        'expand_contractions',
        'expand_missing_delimiter',
        'remove_mentions',
        'remove_hashtags',
        'remove_new_lines',
        'keep_only_alphabet',
        # 'remove_extra_spaces',
        'remove_english_stop_words',
        'lemmatize_noun',
        'correct_typo'
    ],
    'algorithm': 'lda-bert',
    'algorithm_args': {
            'embedding_model': "all-MiniLM-L6-v2",
            'number_topics': 3,
            'top_n_words': 10,
            'gamma': 15 ,
            'random_state': 42
    }     
}

In [11]:
from src import preprocessor
from src.utils import load_documents

docs, labels = load_documents(dataset=configs['dataset'])

if 'preprocessing_funcs' in configs:
    docs = preprocessor.run(data=docs, prep_functions=configs['preprocessing_funcs'])
    
algorithm_args = configs['algorithm_args']
algorithm_args.update(data_name=configs['dataset'],docs=docs,labels=labels)
print(f'Running with {algorithm_args["number_topics"]} topics')

if configs['algorithm'] == 'lda-bert':
    # Encode data with embedding model
    model = SentenceTransformer(algorithm_args['embedding_model'])
    embeddings = model.encode(docs, show_progress_bar=True)
    
    trainer = Trainer(dataset = configs['dataset'],
                      model_name = configs['algorithm'],
                      params = algorithm_args,
                      topk = algorithm_args["top_n_words"],
                      bt_embeddings = embeddings,
                      )
    
    model, df_output_doc_topic, df_output_topic_word = trainer.train()
    
    from src.evaluator import compute_topic_scores
    df_output_topic_word = compute_topic_scores(df_output_doc_topic, df_output_topic_word)

[INFO] Available Preprocessing Functions in the Module:['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_missing_delimiter', 'remove_mentions', 'remove_hashtags', 'keep_only_alphabet', 'remove_new_lines', 'remove_extra_spaces', 'remove_html_tags', 'expand_contractions', 'remove_english_stop_words', 'lemmatize', 'lemmatize_verb', 'lemmatize_noun', 'lemmatize_adjective', 'correct_typo']
[INFO] Preprocessing starting..
[INFO] These string preprocessing methods will be applied to the data in order:
(  'to_lowercase',
   'standardize_accented_chars',
   'remove_url',
   'expand_missing_delimiter',
   'remove_mentions',
   'remove_hashtags',
   'remove_new_lines',
   'keep_only_alphabet')
[INFO] Then, these tokenized preprocessing methods will be applied to the data in order:
(  '__tokenize',
   'expand_contractions',
   'remove_english_stop_words',
   'lemmatize_noun',
   'correct_typo',
   '__glue')
huggingface/tokenizers: The current process just got forked, after paral

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Preprocessing raw texts ...
Preprocessing raw texts. Done!
Clustering embeddings ...
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
Getting vector representations for BERT ...
Getting vector representations for BERT. Done!
Fitting Autoencoder ...


2022-07-20 11:35:37.211782: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-20 11:35:37.211835: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-20 11:35:37.211904: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-2D4JM72S): /proc/driver/nvidia/version does not exist
2022-07-20 11:35:37.213146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Fitting Autoencoder Done!
13/13 [==============================] - 0s 2ms/step
Clustering embeddings. Done!
the words scores are: [[0.03574329813160033, 0.03330625507717303, 0.030056864337936636, 0.025995125913891144, 0.021121039805036556, 0.016246953696181964, 0.012997562956945572, 0.012997562956945572, 0.012185215272136474, 0.011372867587327376], [0.052525252525252523, 0.01616161616161616, 0.01616161616161616, 0.015151515151515152, 0.015151515151515152, 0.014141414141414142, 0.013131313131313131, 0.013131313131313131, 0.010101010101010102, 0.00808080808080808], [0.06908267270668177, 0.03737259343148358, 0.027180067950169876, 0.026047565118912798, 0.010192525481313703, 0.010192525481313703, 0.007927519818799546, 0.007927519818799546, 0.006795016987542469, 0.0056625141562853904]]
and have length: 3


In [12]:
df_output_topic_word

,run_id,method,method_specific_params,dataset,num_given_topics,reduced,topic_num,topic_size,topic_words,word_scores,num_detected_topics,num_final_topics,duration_secs,diversity_unique,diversity_inv_rbo,coherence_npmi,coherence_v,rand_index
0,1658309733,lda-bert,"{'embedding_model': 'all-MiniLM-L6-v2', 'numbe...",crisis_toy,3,False,0,149,"[smoke, debbie, cyclone, wildfire, earthquake,...","[0.03574329813160033, 0.03330625507717303, 0.0...",3,3,10.295372,0.666667,0.629645,-0.30072,0.316743,0.64121
1,1658309733,lda-bert,"{'embedding_model': 'all-MiniLM-L6-v2', 'numbe...",crisis_toy,3,False,1,116,"[flood, wildfire, cyclone, rain, debbie, smoke...","[0.052525252525252523, 0.01616161616161616, 0....",3,3,10.295372,0.666667,0.629645,-0.30072,0.316743,0.64121
2,1658309733,lda-bert,"{'embedding_model': 'all-MiniLM-L6-v2', 'numbe...",crisis_toy,3,False,2,131,"[earthquake, flood, debbie, cyclone, felt, old...","[0.06908267270668177, 0.03737259343148358, 0.0...",3,3,10.295372,0.666667,0.629645,-0.30072,0.316743,0.64121


In [20]:
df_output_doc_topic

,run_id,Document ID,Document,Real Label,Assigned Topic Num,Assignment Score
0,1658231276,0,thereformedcrow nah going go earthquake,earthquake,2,1
1,1658231276,1,think earthquake,earthquake,2,1
2,1658231276,2,uhh else felt earthquake though,earthquake,2,1
3,1658231276,3,bay area nice size earthquake,earthquake,0,1
4,1658231276,4,thought dad farting turn earthquake,earthquake,2,1
...,...,...,...,...,...,...
391,1658231276,391,people keep asking good safe live cyclone debb...,hurricane,0,1
392,1658231276,392,ayyeeee work got cancelled flood thank cyclone...,hurricane,1,1
393,1658231276,393,jetstarairways helpful need change flight due ...,hurricane,0,1
394,1658231276,394,getting hit ex tropical cyclone named debbie r...,hurricane,0,1
